In [1]:
import networkx as nx
from datetime import datetime

class EventGraph:
    def __init__(self):
        self.G = nx.DiGraph()

    def add_event(self, ev: dict):
        """
        Add a new event and its causal/temporal edges.
        """
        eid = ev["id"]
        # 1) add node with all attributes
        self.G.add_node(eid,
                        time=ev["time"],
                        space=ev.get("space"),
                        entities=ev.get("entities"),
                        causation=ev.get("causation"),
                        motivation=ev.get("motivation"),
                        text=ev["text"])
        # 2) causal edge, if any
        parent = ev.get("causation")
        if parent and parent in self.G:
            self.G.add_edge(parent, eid, relation="causal")
        # 3) temporal edges vs. existing events
        t_new = datetime.fromisoformat(ev["time"])
        for other in self.G.nodes():
            if other == eid:
                continue
            t_other = datetime.fromisoformat(self.G.nodes[other]["time"])
            if t_other < t_new:
                self.G.add_edge(other, eid, relation="temporal")
            elif t_other > t_new:
                self.G.add_edge(eid, other, relation="temporal")

# --- Example with given “premise” ---

premise = (
    "Screenwriter moviegoers are those who don't mind being spoiled by spoilers "
    "and even inquire about plot introductions and review all kinds of movies in advance. "
    "This kind of moviegoers pursue the feeling of controlling the development of the plot "
    "and don't like surprises."
)

# Manually extract events from the premise:
events = [
    {
        "id": "e1",
        "time": "2025-05-18T00:00:00",
        "space": None,
        "entities": {"agent": "moviegoers", "patient": "spoilers"},
        "causation": None,
        "motivation": "they seek plot information",
        "text": "Moviegoers don't mind being spoiled by spoilers."
    },
    {
        "id": "e2",
        "time": "2025-05-18T00:01:00",
        "space": None,
        "entities": {"agent": "moviegoers", "patient": "plot introductions"},
        "causation": "e1",
        "motivation": "they want to know the story beforehand",
        "text": "They inquire about plot introductions."
    },
    {
        "id": "e3",
        "time": "2025-05-18T00:02:00",
        "space": None,
        "entities": {"agent": "moviegoers", "patient": "movies"},
        "causation": "e2",
        "motivation": "they like to review in advance",
        "text": "They review all kinds of movies in advance."
    },
    {
        "id": "e4",
        "time": "2025-05-18T00:03:00",
        "space": None,
        "entities": {"agent": "moviegoers", "patient": "plot development"},
        "causation": None,
        "motivation": "they pursue control over the story",
        "text": "They pursue the feeling of controlling the development of the plot."
    },
    {
        "id": "e5",
        "time": "2025-05-18T00:04:00",
        "space": None,
        "entities": {"agent": "moviegoers", "patient": "surprises"},
        "causation": "e4",
        "motivation": "they don't like surprises",
        "text": "They don't like surprises."
    },
]

# Build the graph
eg = EventGraph()
for ev in events:
    eg.add_event(ev)

# Inspect nodes and edges
print("Nodes with attributes:")
for nid, attrs in eg.G.nodes(data=True):
    print(f"{nid}: {attrs}")

print("\nEdges:")
for u, v, d in eg.G.edges(data=True):
    print(f"{u} -> {v}  (relation={d['relation']})")


Nodes with attributes:
e1: {'time': '2025-05-18T00:00:00', 'space': None, 'entities': {'agent': 'moviegoers', 'patient': 'spoilers'}, 'causation': None, 'motivation': 'they seek plot information', 'text': "Moviegoers don't mind being spoiled by spoilers."}
e2: {'time': '2025-05-18T00:01:00', 'space': None, 'entities': {'agent': 'moviegoers', 'patient': 'plot introductions'}, 'causation': 'e1', 'motivation': 'they want to know the story beforehand', 'text': 'They inquire about plot introductions.'}
e3: {'time': '2025-05-18T00:02:00', 'space': None, 'entities': {'agent': 'moviegoers', 'patient': 'movies'}, 'causation': 'e2', 'motivation': 'they like to review in advance', 'text': 'They review all kinds of movies in advance.'}
e4: {'time': '2025-05-18T00:03:00', 'space': None, 'entities': {'agent': 'moviegoers', 'patient': 'plot development'}, 'causation': None, 'motivation': 'they pursue control over the story', 'text': 'They pursue the feeling of controlling the development of the plot.

In [1]:
import re
import networkx as nx
from datetime import datetime, timedelta
from transformers import pipeline

srl = pipeline("text-generation", model="meta-llama/Llama-3.2-1B", cache_dir = '/data3/hg_weight/hg_weight')

def extract_events(text: str,
                   start_time: str = "2025-05-18T00:00:00",
                   interval_minutes: int = 1) -> list:
    """
    입력 텍스트에서 SRL로 동사(이벤트)와 ARG0/ARG1을 뽑아
    이벤트 리스트를 반환.
    time 필드는 start_time에서 interval_minutes 간격으로 자동 할당.
    """
    srl_out = srl(text)[0]              # 첫 번째 문장만 처리
    base_time = datetime.fromisoformat(start_time)
    events = []

    for i, verb in enumerate(srl_out["verbs"]):
        desc = verb["description"]
        # [ARG0: ...], [V: ...], [ARG1: ...] 등을 regex로 추출
        args = dict(re.findall(r"\[(\w+):\s(.*?)\]", desc))

        ev = {
            "id":   f"e{i+1}",
            "time": (base_time + timedelta(minutes=i * interval_minutes))
                        .isoformat(),
            "space":      None,                # 필요 시 NER/문맥분석으로 채워 넣기
            "entities": {
                "agent":   args.get("ARG0"),
                "action":  args.get("V"),
                "patient": args.get("ARG1")
            },
            "causation": None,                 # 추후 인과 추론 로직 추가 가능
            "motivation": None,                # 추후 동기 추출 로직 추가 가능
            "text":       desc
        }
        events.append(ev)

    return events

# 2) EventGraph 클래스: add_event 시 자동으로 temporal/causal 엣지 생성
class EventGraph:
    def __init__(self):
        self.G = nx.DiGraph()

    def add_event(self, ev: dict):
        eid = ev["id"]
        # 1) 노드 추가
        self.G.add_node(eid, **ev)

        # 2) causal 엣지 (ev["causation"]가 있으면)
        parent = ev.get("causation")
        if parent and parent in self.G:
            self.G.add_edge(parent, eid, relation="causal")

        # 3) temporal 엣지: 기존 노드들과 시간 비교
        t_new = datetime.fromisoformat(ev["time"])
        for other, data in self.G.nodes(data=True):
            if other == eid:
                continue
            t_other = datetime.fromisoformat(data["time"])
            if t_other < t_new:
                self.G.add_edge(other, eid, relation="temporal")
            elif t_other > t_new:
                self.G.add_edge(eid, other, relation="temporal")


# 3) 전체 파이프라인 실행 예제
if __name__ == "__main__":
    premise = (
        "Screenwriter moviegoers are those who don't mind being spoiled by spoilers "
        "and even inquire about plot introductions and review all kinds of movies in advance. "
        "This kind of moviegoers pursue the feeling of controlling the development of the plot "
        "and don't like surprises."
    )

    # (1) SRL로 이벤트 추출
    events = extract_events(premise,
                            start_time="2025-05-18T00:00:00",
                            interval_minutes=1)

    # (2) EventGraph 생성 및 이벤트 추가
    eg = EventGraph()
    for ev in events:
        eg.add_event(ev)

    # (3) 결과 확인
    print("=== Extracted Events ===")
    for ev in events:
        print(f"{ev['id']}: {ev['text']}  [{ev['entities']}] @ {ev['time']}")

    print("\n=== Graph Edges ===")
    for u, v, d in eg.G.edges(data=True):
        print(f"{u} -> {v}  (relation={d['relation']})")


model.safetensors:  10%|9         | 241M/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Device set to use cuda:0


ValueError: The following `model_kwargs` are not used by the model: ['cache_dir'] (note: typos in the generate arguments will also show up in this list)